In [1]:
import json
import logging
import time
from pathlib import Path

In [2]:
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    AcceleratorDevice,
    AcceleratorOptions,
    PdfPipelineOptions,
)
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.models.ocr_mac_model import OcrMacOptions
from docling.models.tesseract_ocr_cli_model import TesseractCliOcrOptions
from docling.models.tesseract_ocr_model import TesseractOcrOptions
from docling.models.easyocr_model import  EasyOcrOptions
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
    granite_picture_description,
    smolvlm_picture_description,
)
from docling.datamodel.settings import PageRange

In [3]:
_log = logging.getLogger(__name__)

In [ ]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions, 
    PictureDescriptionApiOptions,PictureDescriptionVlmOptions
)
from docling.document_converter import DocumentConverter, PdfFormatOption

In [ ]:

logging.basicConfig(level=logging.INFO)

input_doc_path = Path("/home/mugesh/Downloads/ijsst07.pdf")

###########################################################################

# The following sections contain a combination of PipelineOptions
# and PDF Backends for various configurations.
# Uncomment one section at the time to see the differences in the output.

# PyPdfium without EasyOCR
# --------------------
# pipeline_options = PdfPipelineOptions()
# pipeline_options.do_ocr = False
# pipeline_options.do_table_structure = True
# pipeline_options.table_structure_options.do_cell_matching = False

# doc_converter = DocumentConverter(
#     format_options={
#         InputFormat.PDF: PdfFormatOption(
#             pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend
#         )
#     }
# )

# PyPdfium with EasyOCR
# -----------------
# pipeline_options = PdfPipelineOptions()
# pipeline_options.do_ocr = True
# pipeline_options.do_table_structure = True
# pipeline_options.table_structure_options.do_cell_matching = True

# doc_converter = DocumentConverter(
#     format_options={
#         InputFormat.PDF: PdfFormatOption(
#             pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend
#         )
#     }
# )

# Docling Parse without EasyOCR
# -------------------------
# pipeline_options = PdfPipelineOptions()
# pipeline_options.do_ocr = False
# pipeline_options.do_table_structure = True
# pipeline_options.table_structure_options.do_cell_matching = True

# doc_converter = DocumentConverter(
#     format_options={
#         InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
#     }
# )

# Docling Parse with EasyOCR
# # ----------------------
# pipeline_options = PdfPipelineOptions()
# pipeline_options.do_ocr = True
# pipeline_options.do_table_structure = True
# pipeline_options.table_structure_options.do_cell_matching = True
# pipeline_options.ocr_options.lang = ["es"]
# pipeline_options.accelerator_options = AcceleratorOptions(
#     num_threads=4, device=AcceleratorDevice.AUTO
# )
# pipeline_options.artifacts_path = Path("/home/mugesh/.cache/docling/models")

# doc_converter = DocumentConverter(
#     format_options={
#         InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
#     }
# )

# Docling Parse with EasyOCR (CPU only)
# ----------------------
# pipeline_options = PdfPipelineOptions()
# pipeline_options.do_ocr = True
# pipeline_options.ocr_options.use_gpu = False  # <-- set this.
# pipeline_options.do_table_structure = True
# pipeline_options.table_structure_options.do_cell_matching = True

# doc_converter = DocumentConverter(
#     format_options={
#         InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
#     }
# )

# Docling Parse with Tesseract
# ----------------------
pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = True
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = True
pipeline_options.ocr_options = EasyOcrOptions()
pipeline_options.artifacts_path = Path("/home/mugesh/.cache/docling/models")
pipeline_options.do_picture_classification = True
pipeline_options.document_timeout = 120

pipeline_options.images_scale = 2.0
pipeline_options.generate_picture_images = True

pipeline_options.accelerator_options = AcceleratorOptions(
    num_threads=4, device=AcceleratorDevice.AUTO
)

doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

# Docling Parse with Tesseract CLI
# ----------------------
# pipeline_options = PdfPipelineOptions()
# pipeline_options.do_ocr = True
# pipeline_options.do_table_structure = True
# pipeline_options.table_structure_options.do_cell_matching = True
# pipeline_options.ocr_options = TesseractCliOcrOptions()

# doc_converter = DocumentConverter(
#     format_options={
#         InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
#     }
# )

# Docling Parse with ocrmac(Mac only)
# ----------------------
# pipeline_options = PdfPipelineOptions()
# pipeline_options.do_ocr = True
# pipeline_options.do_table_structure = True
# pipeline_options.table_structure_options.do_cell_matching = True
# pipeline_options.ocr_options = OcrMacOptions()

# doc_converter = DocumentConverter(
#     format_options={
#         InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
#     }
# )


In [7]:
from docling.datamodel.base_models import ConversionStatus
from docling.datamodel.document import ConversionResult
from docling.datamodel.settings import settings
from docling.document_converter import DocumentConverter

In [8]:
 # Turn on inline debug visualizations:
settings.debug.visualize_layout = True
settings.debug.visualize_ocr = True
settings.debug.visualize_tables = True
settings.debug.visualize_cells = True

# Enable the profiling to measure the time spent
settings.debug.profile_pipeline_timings = True

In [24]:

start_time = time.time()
conv_result = doc_converter.convert(input_doc_path, max_num_pages=100)
end_time = time.time() - start_time

_log.info(f"Document converted in {end_time:.2f} seconds.")


INFO:docling.document_converter:Going to convert document batch...
INFO:docling.pipeline.base_pipeline:Processing document ijsst07.pdf


INFO:docling.document_converter:Finished converting document ijsst07.pdf in 22.69 sec.
INFO:__main__:Document converted in 22.70 seconds.


In [10]:
 # List with total time per document
doc_conversion_secs = conv_result.timings["pipeline_total"].times
doc_conversion_secs

[20.671670527997776]

In [11]:
conv_result.pages

[Page(page_no=0, size=Size(width=595.0, height=842.0), cells=[Cell(id=0, text='10                     ISSN 1473-804x online, 1473-8031 print', bbox=BoundingBox(l=286.8, t=794.79, r=520.2, b=808.092, coord_origin=<CoordOrigin.TOPLEFT: 'TOPLEFT'>)), Cell(id=1, text='E. CASALICCHIO, R.LANCELLOTTI, M.E. POLEGGI: SIMULATION FRAMEWORK', bbox=BoundingBox(l=115.2, t=35.989999999999895, r=480.17999999999995, b=47.05999999999983, coord_origin=<CoordOrigin.TOPLEFT: 'TOPLEFT'>)), Cell(id=2, text='A SIMULATION FRAMEWORK FOR CLUSTER-BASED WEB ', bbox=BoundingBox(l=72.4, t=70.94399999999996, r=527.1, b=88.65599999999995, coord_origin=<CoordOrigin.TOPLEFT: 'TOPLEFT'>)), Cell(id=3, text='SERVICES', bbox=BoundingBox(l=258.0, t=88.644, r=337.48, b=106.356, coord_origin=<CoordOrigin.TOPLEFT: 'TOPLEFT'>)), Cell(id=4, text='EMILIANO CASALICCHIO', bbox=BoundingBox(l=226.3, t=117.4079999999999, r=369.188, b=130.692, coord_origin=<CoordOrigin.TOPLEFT: 'TOPLEFT'>)), Cell(id=5, text='Dipartimento di Informatica,

In [12]:
for page in conv_result.pages:
    print(page.page_no)
    # print(page.cells)
    for j in page.cells:
        print(j.text)

0
10                     ISSN 1473-804x online, 1473-8031 print
E. CASALICCHIO, R.LANCELLOTTI, M.E. POLEGGI: SIMULATION FRAMEWORK
A SIMULATION FRAMEWORK FOR CLUSTER-BASED WEB 
SERVICES
EMILIANO CASALICCHIO
Dipartimento di Informatica, Sistemi e Produzione
Università di Roma 'Tor Vergata'
E-mail: casalicchio@uniroma2.it
RICCARDO LANCELLOTTI
Dipartimento di Iingegneria dell'Informazione
Università di Modena e Reggio Emilia
E-mail:riccardo.lancellotti@unimore.it
MARCO EMILIO POLEGGI
CERN-IT/INFN-CNAF
E-mail: Marco.Emilio.Poleggi@cern.ch
Abstract: 
We propose a simulation framework, namely CWebSim, specifically designed for the performance 
evaluation and capacity planning of cluster-based Web services. A broad variety of Web cluster configurations 
can be simulated through CWebSim. Its modularity permits the definition of different mechanisms, algorithms, 
network topologies and hardware resources. Also, two workload input alternatives are possible: a trace-driven 
mode   and   a   distri

In [13]:
page

Page(page_no=12, size=Size(width=595.0, height=842.0), cells=[Cell(id=0, text='22                     ISSN 1473-804x online, 1473-8031 print', bbox=BoundingBox(l=286.8, t=794.79, r=520.2, b=808.092, coord_origin=<CoordOrigin.TOPLEFT: 'TOPLEFT'>)), Cell(id=1, text='E. CASALICCHIO, R.LANCELLOTTI, M.E. POLEGGI: SIMULATION FRAMEWORK', bbox=BoundingBox(l=115.2, t=35.989999999999895, r=480.17999999999995, b=47.05999999999983, coord_origin=<CoordOrigin.TOPLEFT: 'TOPLEFT'>)), Cell(id=2, text='He acts as reviewer for international conferences, ', bbox=BoundingBox(l=71.0, t=70.88999999999999, r=286.1, b=81.96000000000004, coord_origin=<CoordOrigin.TOPLEFT: 'TOPLEFT'>)), Cell(id=3, text='and   journals:   IEEE   Transaction   on   Software ', bbox=BoundingBox(l=71.0, t=81.9899999999999, r=286.1, b=93.05999999999983, coord_origin=<CoordOrigin.TOPLEFT: 'TOPLEFT'>)), Cell(id=4, text='Engineering   (2007),   Computer   Communication ', bbox=BoundingBox(l=71.0, t=93.09000000000003, r=286.1, b=104.1599

In [14]:
for j in conv_result.timings:
    print(j, conv_result.timings[j].times)

pipeline_total [20.671670527997776]
doc_build [20.402449786000943]
page_init [0.15263519799918868, 0.27451100300095277, 0.464045573000476, 0.3357865750003839, 0.33256045499729225, 0.38977418699869304, 0.34196992800207227, 0.2904602810012875, 0.06597143300314201, 0.2736776799974905, 0.44695193100051256, 0.3476974290024373, 0.11929619199872832]
page_parse [0.16197865299909608, 0.14685985799951595, 0.14315710200025933, 0.14985327099930146, 0.14689194900347502, 0.16251657199973124, 0.14593761200012523, 0.14732652200109442, 0.14154540600065957, 0.13851100000101724, 0.144731035001314, 0.1402149199966516, 0.11322249699878739]
ocr [0.49749456900099176, 0.572976966999704, 0.1434882369976549, 0.3901742070011096, 0.7057588700008637, 0.5048829520019353, 0.5130433800004539, 0.8717996410014166, 1.7912858730014705, 0.7691163409981527, 0.14025425799991353, 0.14611215999684646, 0.14490341000055196]
layout [0.2632066010010021, 0.1264672920005978, 0.11284574000092107, 0.12161350699898321, 0.1312567470013

In [15]:
print("\nDetailed Profiling Summary:")
for stage, timing in conv_result.timings.items():
    times = timing.times  # Expect a list of timing values.
    if isinstance(times, list) and len(times) > 1:
        # Calculate summary statistics for stages with multiple (page-wise) timings.
        avg_time = sum(times) / len(times)
        min_time = min(times)
        max_time = max(times)
        print(f"Stage: {stage}")
        print(f"  Page-wise timings: {times}")
        print(f"  Average: {avg_time:.4f} sec, Min: {min_time:.4f} sec, Max: {max_time:.4f} sec")
    elif isinstance(times, list):
        # Only one timing value present
        print(f"Stage: {stage} -- Time: {times[0]:.4f} sec")
    else:
        # Fallback if timing isn't a list
        print(f"Stage: {stage} -- Time: {times:.4f} sec")


Detailed Profiling Summary:
Stage: pipeline_total -- Time: 20.6717 sec
Stage: doc_build -- Time: 20.4024 sec
Stage: page_init
  Page-wise timings: [0.15263519799918868, 0.27451100300095277, 0.464045573000476, 0.3357865750003839, 0.33256045499729225, 0.38977418699869304, 0.34196992800207227, 0.2904602810012875, 0.06597143300314201, 0.2736776799974905, 0.44695193100051256, 0.3476974290024373, 0.11929619199872832]
  Average: 0.2950 sec, Min: 0.0660 sec, Max: 0.4640 sec
Stage: page_parse
  Page-wise timings: [0.16197865299909608, 0.14685985799951595, 0.14315710200025933, 0.14985327099930146, 0.14689194900347502, 0.16251657199973124, 0.14593761200012523, 0.14732652200109442, 0.14154540600065957, 0.13851100000101724, 0.144731035001314, 0.1402149199966516, 0.11322249699878739]
  Average: 0.1448 sec, Min: 0.1132 sec, Max: 0.1625 sec
Stage: ocr
  Page-wise timings: [0.49749456900099176, 0.572976966999704, 0.1434882369976549, 0.3901742070011096, 0.7057588700008637, 0.5048829520019353, 0.5130433

In [16]:
from docling_core.types.doc import (
    DoclingDocument,
    NodeItem,
    PictureClassificationClass,
    PictureClassificationData,
    PictureItem,
)

In [17]:
for element, _level in conv_result.document.iterate_items():
        if isinstance(element, PictureItem):
            print(
                f"The model populated the `data` portion of picture {element.self_ref}:\n{element.annotations}"
            )

The model populated the `data` portion of picture #/pictures/0:
[PictureClassificationData(kind='classification', provenance='DocumentPictureClassifier', predicted_classes=[PictureClassificationClass(class_name='other', confidence=0.8954629302024841), PictureClassificationClass(class_name='logo', confidence=0.08827494084835052), PictureClassificationClass(class_name='screenshot', confidence=0.014364967122673988), PictureClassificationClass(class_name='bar_chart', confidence=0.0008396346238441765), PictureClassificationClass(class_name='stamp', confidence=0.00025959828053601086), PictureClassificationClass(class_name='flow_chart', confidence=0.00024804085842333734), PictureClassificationClass(class_name='signature', confidence=0.0001906414981931448), PictureClassificationClass(class_name='pie_chart', confidence=0.00011390967847546563), PictureClassificationClass(class_name='chemistry_molecular_structure', confidence=6.316352664725855e-05), PictureClassificationClass(class_name='line_cha

In [18]:
conv_result.assembled.elements

[TextElement(label=<DocItemLabel.PAGE_FOOTER: 'page_footer'>, id=6, page_no=0, cluster=Cluster(id=6, label=<DocItemLabel.PAGE_FOOTER: 'page_footer'>, bbox=BoundingBox(l=286.8, t=794.79, r=520.2, b=808.092, coord_origin=<CoordOrigin.TOPLEFT: 'TOPLEFT'>), confidence=0.8682352900505066, cells=[Cell(id=0, text='10                     ISSN 1473-804x online, 1473-8031 print', bbox=BoundingBox(l=286.8, t=794.79, r=520.2, b=808.092, coord_origin=<CoordOrigin.TOPLEFT: 'TOPLEFT'>))], children=[]), text='10                     ISSN 1473-804x online, 1473-8031 print'),
 TextElement(label=<DocItemLabel.PAGE_HEADER: 'page_header'>, id=18, page_no=0, cluster=Cluster(id=18, label=<DocItemLabel.PAGE_HEADER: 'page_header'>, bbox=BoundingBox(l=115.2, t=35.989999999999895, r=480.17999999999995, b=47.05999999999983, coord_origin=<CoordOrigin.TOPLEFT: 'TOPLEFT'>), confidence=0.7155863642692566, cells=[Cell(id=1, text='E. CASALICCHIO, R.LANCELLOTTI, M.E. POLEGGI: SIMULATION FRAMEWORK', bbox=BoundingBox(l=115

In [19]:
from docling_core.transforms.chunker.hybrid_chunker import HybridChunker

In [20]:

chunker = HybridChunker()
chunk_iter = chunker.chunk(dl_doc=conv_result.document)

Token indices sequence length is longer than the specified maximum sequence length for this model (543 > 512). Running this sequence through the model will result in indexing errors


In [21]:
for chunk in chunk_iter:
    print(chunk.text)

    print([j.prov[0].page_no for j in chunk.meta.doc_items])
    print("-"*100)
    # break

Dipartimento di Informatica, Sistemi e Produzione
Università di Roma 'Tor Vergata'
E-mail: casalicchio@uniroma2.it
[1, 1, 1]
----------------------------------------------------------------------------------------------------
Dipartimento di Iingegneria dell'Informazione
Università di Modena e Reggio Emilia
E-mail:riccardo.lancellotti@unimore.it
[1, 1, 1]
----------------------------------------------------------------------------------------------------
CERN-IT/INFN-CNAF
E-mail: Marco.Emilio.Poleggi@cern.ch
Abstract: We propose a simulation framework, namely CWebSim, specifically designed for the performance evaluation and capacity planning of cluster-based Web services. A broad variety of Web cluster configurations can be simulated through CWebSim. Its modularity permits the definition of different mechanisms, algorithms, network topologies and hardware resources. Also, two workload input alternatives are possible: a trace-driven mode   and   a   distribution-driven   mode   that   e

In [22]:
##########################################################################

## Export results
output_dir = Path("scratch")
output_dir.mkdir(parents=True, exist_ok=True)
doc_filename = conv_result.input.file.stem

# Export Deep Search document JSON format:
with (output_dir / f"{doc_filename}.json").open("w", encoding="utf-8") as fp:
    fp.write(json.dumps(conv_result.document.export_to_dict()))

# Export Text format:
with (output_dir / f"{doc_filename}.txt").open("w", encoding="utf-8") as fp:
    fp.write(conv_result.document.export_to_text())

# Export Markdown format:
with (output_dir / f"{doc_filename}.md").open("w", encoding="utf-8") as fp:
    fp.write(conv_result.document.export_to_markdown())

# Export Document Tags format:
with (output_dir / f"{doc_filename}.doctags").open("w", encoding="utf-8") as fp:
    fp.write(conv_result.document.export_to_document_tokens())